In [3]:
from bs4 import BeautifulSoup
import string
import requests
import pandas as pd
import re
import pickle

from fake_useragent import UserAgent
ua = UserAgent()

In [4]:
def remove_punctuation(x):
    x = str(x)
    return x.translate(str.maketrans({a:None for a in string.punctuation}))

In [5]:
def get_soup(url, timeout=5):
    headers  = {'User-Agent':ua.random}
    try:
        response = requests.get(url,headers=headers)
    except:
        print("FAILED "+ url)
        return 0
    attempts = 0
    while(not response.ok):
            print((url+' failed with code: '+str(response.status_code)))
            if attempts > timeout:
                print(url+' failed with code: '+str(response.status_code))
                return BeautifulSoup('','lxml')
            response = requests.get(url)
            attempts += 1
    page = response.text
    soup = BeautifulSoup(page,'lxml')
    return soup

In [ ]:
# Get all urls for each beer style

url = 'http://www.beeradvocate.com/beer/style/'
soup = get_soup(url)

beer_styles = {}
for style in soup.find('table').find_all('a'):
    beer_styles[style.get_text()] = style['href']

In [ ]:
ba_url = 'http://www.beeradvocate.com'
style_suffix = '?sort=revsD&start=0'
soup = get_soup(ba_url+style_url+style_suffix)

In [ ]:
int(re.findall(r'(?<=\(out of )\d*',soup.find('tr').get_text())[0])

In [ ]:
beer_styles.items()[0]

In [ ]:
# Pulls the name and url to all beers with more than 25 'hads'

ba_url = 'http://www.beeradvocate.com'
style_suffix = '?sort=revsD&start='
columns = ['name','url']
temp = {}
for style in beer_styles.items():
    print(url)
    url = ba_url+style[1]+style_suffix
    soup = get_soup(url)
    num_beers = int(re.findall(r'(?<=\(out of )\d*',soup.find('tr').get_text())[0])
    print num_beers
    min_beer = False
    for i in range(num_beers//50):
        if min_beer:
                break
        url = ba_url+style[1]+style_suffix+str(i*50)
        soup = get_soup(url)
        for row in soup.find_all('tr')[3:-1]:
            cells = row.find_all('td')
            if int(cells[4].get_text().replace(',','')) < 25:
                min_beer = True
                break
            temp[cells[0].find('a').get_text()] = cells[0].find('a')['href']

In [7]:
# Pulls the name and url to all beers with more than 25 'hads'

ba_url = 'http://www.beeradvocate.com'
style_suffix = '?sort=revsD&start='
columns = ['name','url']
temp = {}

url = ba_url+'/beer/style/149/'+style_suffix
soup = get_soup(url)
num_beers = int(re.findall(r'(?<=\(out of )\d*',soup.find('tr').get_text())[0])
print num_beers
min_beer = False
for i in range(num_beers//50):
    if min_beer:
            break
    url = ba_url+'/beer/style/149/'+style_suffix+str(i*50)
    soup = get_soup(url)
    for row in soup.find_all('tr')[3:-1]:
        cells = row.find_all('td')
        if int(cells[4].get_text().replace(',','')) < 25:
            min_beer = True
            break
        temp[cells[0].find('a').get_text()] = cells[0].find('a')['href']
        
temp

538


{u'1554 Black Lager': '/beer/profile/192/111828/',
 u'3 Best Friends': '/beer/profile/219/116077/',
 u'Alhambra Negra': '/beer/profile/9262/22584/',
 u'Asahi Dry Black': '/beer/profile/716/89545/',
 u'B.B. Dark Bohemia Beer - 1795 Original Czech Dark Lager': '/beer/profile/303/37361/',
 u'Baltika #4 Original (Dark)': '/beer/profile/401/2235/',
 u'Beerlao Dark': '/beer/profile/2970/27607/',
 u'Bernard Cerne': '/beer/profile/2055/21521/',
 u'Big Rock Honey Brown Lager': '/beer/profile/391/11175/',
 u'Black Lager (\u010cern\xe9 Pivo)': '/beer/profile/22723/76132/',
 u'Black Licorice Lager': '/beer/profile/9629/37147/',
 u'Bohemia': '/beer/profile/301/7078/',
 u'Bohemia Regent Lager Dark': '/beer/profile/7366/14349/',
 u'Bony Fingers': '/beer/profile/763/20446/',
 u'Brick Waterloo Dark Lager': '/beer/profile/416/5196/',
 u'Budweiser Budvar Czech Dark Lager': '/beer/profile/304/35967/',
 u'California Black Beer': '/beer/profile/8818/40370/',
 u'Celestial Meridian Cascadian Dark Lager': '/be

In [ ]:
pickle.dump(temp,open('beer_list.pkl','wb'))

In [ ]:
beer_urls = pickle.load(open('beer_list.pkl','rb'))

In [ ]:
columns = ['name','url']
beers = pd.DataFrame(columns=columns)

beers['name'] = beer_urls.keys()
beers['url']  = beer_urls.values()
pd.to_pickle(beers,'beers.pkl')

In [8]:
beers = pd.read_pickle('beers.pkl')

In [9]:
beers[beers.name == 'Death & Taxes Black Beer']

,name,url
2299,Death & Taxes Black Beer,/beer/profile/763/2306/


In [18]:
def get_beer_soup(url):
    ba_url = 'http://www.beeradvocate.com'
    url = ba_url+url
    soup = get_soup(url)
    return soup

In [20]:
import sys
sys.setrecursionlimit(100000000)

In [31]:
for i in tqdm(range(0,beers.shape[0],230)):
    temp = beers.iloc[i:i+230-1,:]
    temp['soup'] = temp.url.map(get_beer_soup)
    temp.to_pickle('beer_soup_'+str(i+230-1)+'.pkl')

  0%|          | 0/89 [00:00<?, ?it/s]

http://www.beeradvocate.com/beer/profile/26676/104914/ failed with code: 403
http://www.beeradvocate.com/beer/profile/2055/13583/ failed with code: 403
http://www.beeradvocate.com/beer/profile/27021/124612/ failed with code: 403


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
  1%|          | 1/89 [02:04<3:02:00, 124.10s/it]

http://www.beeradvocate.com/beer/profile/16773/104438/ failed with code: 403
http://www.beeradvocate.com/beer/profile/395/35378/ failed with code: 403


  2%|▏         | 2/89 [04:09<3:00:19, 124.36s/it]

...

 97%|█████████▋| 86/89 [3:04:55<06:19, 126.40s/it]

http://www.beeradvocate.com/beer/profile/30746/90736/ failed with code: 500
http://www.beeradvocate.com/beer/profile/24299/92753/ failed with code: 403


 99%|█████████▉| 88/89 [3:09:07<02:05, 125.93s/it]

http://www.beeradvocate.com/beer/profile/17980/64545/ failed with code: 403
http://www.beeradvocate.com/beer/profile/25959/70761/ failed with code: 403


100%|██████████| 89/89 [3:11:12<00:00, 125.82s/it]


In [ ]:
temp = beers.iloc[229:beers.shape[0]:230]
temp['soup'] = temp.url.map(get_beer_soup)

In [21]:
temp.head()

,name,url,soup
229,Franconia Amber,/beer/profile/17033/43115/,"<!DOCTYPE html> <html class=""Public NoJs Logge..."
459,Big American Stout,/beer/profile/26824/113395/,"<!DOCTYPE html> <html class=""Public NoJs Logge..."
689,Love's Armor,/beer/profile/28019/90275/,"<!DOCTYPE html> <html class=""Public NoJs Logge..."
919,Deadeye Jack Porter,/beer/profile/19126/47303/,"<!DOCTYPE html> <html class=""Public NoJs Logge..."
1149,Peach Lambic,/beer/profile/1170/45334/,"<!DOCTYPE html> <html class=""Public NoJs Logge..."


In [22]:
temp.to_pickle('beer_soup_missing.pkl')

In [14]:
from tqdm import tqdm
for i in tqdm(range(229,beers.shape[0],230)):
    print(i)

100%|██████████| 89/89 [00:00<00:00, 46279.82it/s]

229
459
689
919
1149
1379
1609
1839
2069
2299
2529
2759
2989
3219
3449
3679
3909
4139
4369
4599
4829
5059
5289
5519
5749
5979
6209
6439
6669
6899
7129
7359
7589
7819
8049
8279
8509
8739
8969
9199
9429
9659
9889
10119
10349
10579
10809
11039
11269
11499
11729
11959
12189
12419
12649
12879
13109
13339
13569
13799
14029
14259
14489
14719
14949
15179
15409
15639
15869
16099
16329
16559
16789
17019
17249
17479
17709
17939
18169
18399
18629
18859
19089
19319
19549
19779
20009
20239
20469


In [ ]:
beer_reviews = {}
ba_url = 'http://www.beeradvocate.com'

for beer in beers.items():
    soup = get_soup(ba_url+beer[1])
    print(beer[0])
    beer_reviews[beer[0]] = get_beer_reviews(soup)